In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score
from .image_classification.scratch_models import model_generator
import argparse
import os
import time
import pandas as pd
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

%matplolib inline

In [ ]:
models = ['densenet', 
        'resnet', 
        'xceptionnet', 
        'mobilenet', 
        'efficientnetv2s', 
        'efficientnetv2m', 
        'efficientnetv2l', 
        'efficientnetv1']

In [ ]:
SHAPE = (256, 256, 3)
EPOCHS = 20
BATCH_SIZE = 32
CLASS_MODE = 'sparse'
OPTIM_TYPE = 'adam'
CLASSES = 25

# Initialize Variables 
# TODO: change the paths
TRAIN_IMG_DIR = os.path.join('data/train')
VALID_IMG_DIR = os.path.join('data/validate')
TEST_IMG_DIR = os.path.join('data/test')

# Initialize Data Generators
# Different Augmentations can also be applied on data
# Image Augmentation to be integrated
# Basic Preprocessing included as of now
train_data_generator = ImageDataGenerator(rescale=1/255.)
validation_data_generator = ImageDataGenerator(rescale=1/255.)

# Get Training, Testing and Validation Data
train_data = train_data_generator.flow_from_directory(
                        TRAIN_IMG_DIR,
                        target_size=SHAPE[:2],
                        batch_size=BATCH_SIZE,
                        class_mode=CLASS_MODE)

valid_data = validation_data_generator.flow_from_directory(
                        VALID_IMG_DIR,
                        target_size=SHAPE[:2],
                        batch_size=BATCH_SIZE//2,
                        class_mode=CLASS_MODE)

print("Training Data Indices: ", train_data.class_indices)
print("Validation Data Indices: ", valid_data.class_indices)

In [ ]:
def plot_training_curve(history):
    # Check how loss & accuracy evolved
    epoch_loss = history.history['loss']
    epoch_val_loss = history.history['val_loss']
    epoch_mae = history.history['acc']
    epoch_val_mae = history.history['val_acc']

    # Learning curve
    plt.figure(figsize=(20,6))
    plt.subplot(1,2,1)
    plt.plot(range(0,len(epoch_loss)), epoch_loss, 'b-', linewidth=2, label='Train Loss')
    plt.plot(range(0,len(epoch_val_loss)), epoch_val_loss, 'r-', linewidth=2, label='Val Loss')
    plt.title('Loss on train & validation datasets over epochs')
    plt.legend(loc='best')

    plt.subplot(1,2,2)
    plt.plot(range(0,len(epoch_mae)), epoch_mae, 'b-', linewidth=2, label='Train Accuracy')
    plt.plot(range(0,len(epoch_val_mae)), epoch_val_mae, 'r-', linewidth=2,label='Val Accuracy')
    plt.title('Accuracy on train & validation datasets over epochs')
    plt.legend(loc='best')

    plt.show()

In [ ]:
# test function
def testing(TEST_IMG_DIR, MODEL_NAME):

    # Load test data
    test_data_generator = ImageDataGenerator(rescale=1/255.)
    test_data = test_data_generator.flow_from_directory(
                            TEST_IMG_DIR,
                            target_size=SHAPE,
                            batch_size=BATCH_SIZE,
                            class_mode=CLASS_MODE,
                            shuffle=False
                            )

    print("Testing Data Indices: ", test_data.class_indices)

    # Load model and calculate metrics
    if os.path.exists(f'image_classification/Models/binary_classification/{MODEL_NAME}_model'):
        model = load_model(f"image_classification/Models/binary_classification/{MODEL_NAME}_model")
        print(f'Successfully loaded {MODEL_NAME}')

        images = len(test_data)

        # Prediction start time
        start_time = time.time()
        predictions = model.predict(x=test_data)
        end_time = time.time()

        # Convert predictions to disease classes
        classes = []
        for pred in predictions:
            cls = list(pred).index(pred.max())
            classes.append(cls)
                
        accuracy = accuracy_score(list(test_data.classes), classes)
        
        f1_global = f1_score(list(test_data.classes), classes, average="micro")
        f1_individual = f1_score(list(test_data.classes), classes, average="macro")
        
        precision_global = precision_score(list(test_data.classes), classes, average="micro")
        precision_individual = precision_score(list(test_data.classes), classes, average="macro")
        
        recall_global = recall_score(list(test_data.classes), classes, average="micro")
        recall_individual = recall_score(list(test_data.classes), classes, average="macro")
        
        total_infer_time = (end_time-start_time)*1e3
        
        infer_time = ((end_time-start_time)/images)*1e3

        print('Model Name: ', MODEL_NAME)
        print('---------------------------------------------------')
        print('Testing Scores: ')
        print('---------------------------------------------------')
        print('Classification Report')
        print(classification_report(list(test_data.classes), classes))
        print('---------------------------------------------------')
        print('Accuracy: ', accuracy)
        print('---------------------------------------------------')
        print('F1 Score Global: ', f1_global)
        print('---------------------------------------------------')
        print('F1 Score Individual: ', f1_individual)
        print('---------------------------------------------------')
        print('Precision Global: ', precision_global)
        print('---------------------------------------------------')
        print('Precision Individual: ', precision_individual)
        print('---------------------------------------------------')
        print('Recall Global: ', recall_global
        print('---------------------------------------------------')
        print('Recall Individual: ', recall_individual
        print('---------------------------------------------------')
        print('Total Inference Time in ms: ', total_infer_time)
        print('Inference time for one image in ms: ', infer_time)
        print('---------------------------------------------------')

        record = {'model':[MODEL_NAME], 
                  'accuracy':[accuracy], 
                  'f1_score_global':[f1_global], 
                  'f1_score_individual':[f1_individual],
                  'precision_global':[precision_global], 
                  'precision_individual':[precision_individual], 
                  'recall_global':[recall_global],
                  'recall_individual':[recall_individual], 
                  'total_pred_time':[total_infer_time], 
                  'pred_time':[infer_time]}
        
        return record
    else:
        raise ValueError(f"No model named {MODEL_NAME} found!")

In [ ]:
def log_result(result):
    csv_path = 'image_classification/results/multiclass_classification_results.csv'
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df_new = pd.DataFrame(result)
        df = pd.concat([df, df_new], axis=0)
        df.to_csv(csv_path)
    else:
        df_new = pd.DataFrame(result)
        df.to_csv(csv_path)
    
    return "Results logged successfully!!"

In [ ]:
# Constants and CALLBACKS
# Model 1
MODEL_NAME = ""
steps_per_epoch = len(train_data)//BATCH_SIZE
checkpoint_path = f"image_classification/ckpt/multiclass_classification/{MODEL_NAME}/best_model"

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True)
tensorboard = TensorBoard(log_dir='image_classification/logs/multiclass_classification/{MODEL_NAME}')
loss = BinaryCrossentropy()
optim = Adam(learning_rate=10e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1,
                              patience=5, 
                              min_lr=10e-12, 
                              verbose=1)
callback_collection = [checkpoint, tensorboard, reduce_lr]

In [ ]:
# Compile Model and Train
model_1 = model_generator(MODEL_NAME, CLASSES)
model_1.compile(optimizer=optim, loss=loss, metrics='accuracy')
history = model_1.fit(
            train_data,
            validation_data = valid_data,
            epochs = EPOCHS,
            callbacks=callback_collection
)

# Save model
model_1.save(f'image_classification/Models/multiclass_classification/{MODEL_NAME}_model')

In [ ]:
# testing
result_1 = testing(TEST_IMG_DIR, MODEL_NAME)
print(log_result(result_1))